In [3]:
import torch

In [15]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments
device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")
# Load the GPT-2 model and tokenizer
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name).to(device)

# Load your dataset
def load_dataset(file_path, tokenizer, block_size=128):
    dataset = TextDataset(
        tokenizer=tokenizer,
        file_path=file_path,
        block_size=block_size
    )
    return dataset

# Define the data collator (for dynamically creating batches)
def data_collator(tokenizer):
    return DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=False  # Masked Language Model is False because GPT-2 is a causal LM
    )

# Training arguments
training_args = TrainingArguments(
    output_dir="./gpt2-finetuned",       # Output directory for saving checkpoints
    overwrite_output_dir=True,           # Overwrite output directory
    num_train_epochs=3,                  # Number of training epochs
    per_device_train_batch_size=2,       # Batch size
    save_steps=500,                      # Save checkpoint every 500 steps
    save_total_limit=2,                  # Limit the number of saved checkpoints
    prediction_loss_only=True,           # Only calculate loss (no eval metrics)
    logging_dir='./logs',                # Directory for logs
    logging_steps=100,
    learning_rate=5e-5,warmup_steps=100,)

# Load your dataset
file_path = 'output.txt'  # Specify your .txt file path
dataset = load_dataset(file_path, tokenizer)

# Create the Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator(tokenizer),
    train_dataset=dataset
)

# Fine-tune the model
trainer.train()

# Save the final model
trainer.save_model('./gpt2-finetuned')
tokenizer.save_pretrained('./gpt2-finetuned')



/data/home/arunkumar12/miniconda3/envs/torch/lib/python3.9/site-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
/data/home/arunkumar12/miniconda3/envs/torch/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss
100,4.034500
200,3.579000
300,3.443000
400,3.356500
500,3.292600
600,3.270300
700,3.225600
800,3.197600
900,3.194900
1000,3.152500


/data/home/arunkumar12/miniconda3/envs/torch/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/data/home/arunkumar12/miniconda3/envs/torch/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/data/home/arunkumar12/miniconda3/envs/torch/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/data/home/arunkumar12/miniconda3/envs/torch/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and re

('./gpt2-finetuned/tokenizer_config.json',
 './gpt2-finetuned/special_tokens_map.json',
 './gpt2-finetuned/vocab.json',
 './gpt2-finetuned/merges.txt',
 './gpt2-finetuned/added_tokens.json')

In [16]:
# If your tokenizer doesn't have a padding token, you might need to add it
tokenizer.pad_token = tokenizer.eos_token


In [17]:
# Define the input prompt
input_prompt = "Krishna said to arjuna"

# Encode the input prompt
input_ids = tokenizer.encode(input_prompt, return_tensors='pt')
input_ids=input_ids.to(device)


In [18]:
# Generate text using the model
model=model.to(device)
output = model.generate(
    input_ids,
    max_length=100,           # Maximum number of tokens to generate
    num_return_sequences=2,    # Number of sequences to generate
    no_repeat_ngram_size=2,    # Avoid repeating n-grams
    do_sample=True,            # Enable sampling
    top_k=50,                  # Use top-k sampling
    top_p=0.95,                # Use nucleus sampling
    temperature=0.7            # Controls the creativity of the output (lower = less random)
)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In [19]:
# Decode the generated tokens to text
for i in range(len(output)):
    generated_text = tokenizer.decode(output[i], skip_special_tokens=True)


    print("---------------------------------------------------------------------")
    print(generated_text)


Krishna said to arjuna, 'O foremost of Rishis, it is said that the Brahmanas are of high birth and mighty-
armed and possessed of great energy, and are the foremost ones of the celestials. They are great in number, they are possessed
of great wisdom, the sons of Draupadi are endued with great prowess, great intelligence, of unrivalled prowess. O thou of
the celestial race, I say to thee
Krishna said to arjuna, 'O bull among men, when I shall see thy son, I will slay him. There is no other man in this world who is equal to him in
consequence of his prowess.'
"Arjunas said, "O thou foremost of men and the foremost amongst men in the world, thou wilt have to slay me in battle. I tell thee truly, O
possessed of great wisdom, that I am even like
